In [18]:
from sage.all import *
from sympy import Matrix
import numpy as np
import random
import math
from itertools import combinations
from sympy import Matrix, Rational, mod_inverse
class Attack:
    def __init__(self, m,t, p, l):
        self.m = int(m)
        self.t = int(t)
        self.n = int(2**(m))
        self.k = int(self.n - self.m * self.t )
        self.p = int(p)
        self.l = int(l)
        self.modulus = 2
    def is_irreducible(self,poly, field):
        return poly.is_irreducible()
    def generate_random_goppa_code(self):
        F = FiniteField(2^ (self.m))
        R.<x>= PolynomialRing(F)
        dim = 0
        iter_err = 0
        while dim != int(self.n - self.m * self.t):
            while iter_err < 2^10:
                iter_err += 1
                g = R.irreducible_element(self.t)
                if g.is_irreducible():
                    Support = [a for a in F.list() if g(a)!= 0]
                    np.random.shuffle(Support)
                    C = codes.GoppaCode(g,Support[:self.n])
                    dim = C.dimension()
                    if dim == self.n - self.m * self.t:
                        parity_matrices = C.parity_check_matrix()
                        return C.generator_matrix()
        return " Failure during the generation of Goppa code. Try it again"
    def Generate_S_invertible(self):
        condition = True  
        while condition:
            S = np.mod(np.random.permutation( self.k * self.k ).reshape(self.k,self.k ),2)
            if np.linalg.det(S)!=0:
                condition = False
        return  S
    def generate_permutation_matrix(self):
        self.permutation_matrix = np.eye(self.n, dtype=int)[np.random.permutation(self.n)] 
        return self.permutation_matrix
    def Transform_echelon_Matrix_mod2(self,x):
        numer, denom = x.as_numer_denom()
        return numer*mod_inverse(denom,self.modulus) % self.modulus
    def is_echelon_form(self,matrix_rref):
        diagonal = list()
        for i in range(self.k):
            diagonal.append( matrix_rref[i,i] )
        if sum(diagonal) == self.k:
            return True
        else:
            return False
    def echelon_form(self,matrix):
        #Find row-reduced echolon form of matrix modulo 2:
        matrix_rref = matrix.rref( iszerofunc = lambda x: x % self.modulus ==0)[0].applyfunc(lambda x:  self.Transform_echelon_Matrix_mod2(x)   )
        row, cols =  matrix_rref.shape  
        if self.is_echelon_form(matrix_rref):
            return matrix_rref, True
        else:
            return matrix_rref,False
    def generate_public_key(self):
        control = True
        iteration = 0
        while control:
            iteration  += 1
            S = self.Generate_S_invertible() 
            P = self.generate_permutation_matrix( )
            Public_key = np.mod(np.dot( np.mod( np.dot(S,np.array(self.generate_random_goppa_code()) )  , 2)  , P), 2).astype(int)
            sympy_matrix = Matrix(Public_key)
            echelon_matrix, control =  self.echelon_form(sympy_matrix)
            if control == True:
                control = False
                return Public_key
            else:
                control = True
    def verify_inverse_public_key(self, random_index):
        matrix_I = np.array( self.public_key )[:, random_index].astype(int)
        self.k   = self.public_key[0]
        return np.linalg.det(matrix_I) != 0
    def GaussianEliminationMod2(self,G):
        for j in range(self.k):
            pivot_i = -1
            for i in range(j,self.k):
                if G[i,j] == 1:
                    pivot_i = i
                    break
            if pivot_i == -1:
                raise ValueError("first k columns do not have full rank")
            assert(G[pivot_i,j] == 1)
            if pivot_i != j:
                tmp = G[pivot_i,:].copy()
                G[pivot_i,:] = G[j,:].copy()
                G[j,:] = tmp
            for i in range(self.k):
                if i != j and G[i,j]==1:
                    G[i,:] = np.mod( G[i,:] + G[j,:], 2 )
        return G
    def NextConfiguration(self, G, z, sigma):
        permutation_gives_fullrank_block = False
        while not permutation_gives_fullrank_block:
            theta = random.sample([i for i in range(self.n)], self.n)
            try:
                G = self.GaussianEliminationMod2(G[:, theta])
            except:
                pass
            else:
                permutation_gives_fullrank_block = True
        zz = z[theta]
        z = np.mod(zz + zz[:self.k].dot(G), 2)  
        sigma = sigma[theta]
        return G, z, sigma, theta[:self.k]
    def DecodeError_McEliece_information(self,G):
        assert(len(self.ciphertext) == self.n)
        z = self.ciphertext.copy()
        sigma = np.arange(self.n)
        while True:
            G, z, sigma, information_set_decoding = self.NextConfiguration(G, z, sigma)
            assert(all(z[:self.k]==0))
            if sum(z[self.k:]) == self.t:
                break
        e = np.zeros(self.n, dtype = np.int32)
        for i, loc in enumerate(sigma):
            e[loc] = z[i] 
        return e, information_set_decoding
    def DecodeError_LeeBrickell(self,G):
        assert(len(self.ciphertext) == self.n)
        z = self.ciphertext.copy()
        sigma = np.arange(self.n)
        found = False
        while True:
            G, z, sigma,information_set_decoding = self.NextConfiguration(G, z, sigma)
            assert(all(z[:self.k] == 0))
            for r in range(p+1):
                for m_support in list(combinations([i for i in range(self.k)], r)):
                    m = np.zeros(self.k, dtype=np.int32)
                    for loc in m_support:
                        m[loc] = 1
                    q = np.mod(z + m.dot(G), 2)
                    assert(sum(q) == r + sum(q[self.k:]))
                    if sum(q[self.k:]) == self.t - r:
                        found = True
                        break
                if found:
                    break
            if found:
                break
        e = np.zeros(self.n, dtype = np.int32)
        for i, loc in enumerate(sigma):
            e[loc] = q[i]  
        return e, information_set_decoding
    def DecodeError_Leon(self,G):
        assert(len(self.ciphertext) == self.n)
        z = self.ciphertext.copy()
        sigma = np.arange(self.n)
        n_loops = 0
        found = False
        while True:
            n_loops += 1
            G, z, sigma, information_set_decoding = self.NextConfiguration(G, z, sigma)
            assert(all(z[: self.k]==0))
            for r in range(self.p+1):
                for m_support in list(combinations([i for i in range(self.k)], r)):
                    m = np.zeros(self.k, dtype = np.int32)
                    for loc in m_support:
                        m[loc] = 1
                    q = np.mod(z + m.dot(G), 2)
                    if sum(q[self.k : self.k + self.l]) <= self.p - r:
                        assert(sum(q) == r + sum(q[self.k : self.k + self.l]) + sum(q[self.k + self.l:]))
                        if sum(q[self.k + self.l:]) == self.t - r - sum(q[self.k : self.k + self.l]):
                            found = True
                            break
                if found:
                    break
            if found:
                break
        e = np.zeros(self.n, dtype = np.int32)
        for i, loc in enumerate(sigma):
            e[loc] = q[i]  
        return e, information_set_decoding
    def encrypt(self,G):
        self.original_message = np.random.randint(low = 0, high = 1 + 1 , size = self.k ).astype(np.int32)   
        error_locations = random.sample([i for i in range(self.n)], self.t)
        self.original_error = np.zeros(shape = self.n, dtype = np.int32)
        for loc in error_locations:
            self.original_error[loc] = 1
        self.original_codeword   =  np.mod( np.dot(self.original_message,G),2 )
        self.ciphertext = np.bitwise_xor(self.original_codeword ,self.original_error)
        print("Original message:  \t", self.original_message)
        print("ciphertext:   \t\t" , self.ciphertext)
        print("\n")
    def decrypt_McEliece(self,G):
        self.error_McEliece, self.I_McEliece   = self.DecodeError_McEliece_information(G)
        if np.array_equal( self.original_error,self.error_McEliece):
            return self.error_McEliece, self.I_McEliece 
        else:
            print("McEliece   FAILURE!!!!!!!")
    def decrypt_Leon(self,G):
        self.error_Leon, self.I_Leon       = self.DecodeError_Leon(G )
        if np.array_equal( self.original_error,self.error_Leon):
            return self.error_Leon, self.I_Leon
        else:
            print("Leon   FAILURE!!!!!!!")
    def decrypt_LeeBrickell(self, G):
        self.error_LeeBrickell, self.I_LeeBrickell    = self.DecodeError_LeeBrickell(G)
        if np.array_equal( self.original_error,self.error_LeeBrickell): 
            return self.error_LeeBrickell, self.I_LeeBrickell
        else:
            print("Lee Brickell   FAILURE!!!!!!!")
    def is_invertible_mod_2(self,matrix):
        if not isinstance(matrix, Matrix):
            matrix = Matrix(matrix)  # Convert to a SymPy Matrix if it's not already.
        try:
            det = matrix.det()  # Compute the determinant of the matrix
            return det % 2 == 1  # Check if the determinant is 1 modulo 2
        except Exception as e:
            print("Error calculating determinant:", e)
            return False
    def validate(self):
        recover = { "McEliece":[], "Leon":[],  "Lee Brickell":[] }
        for i in ["McEliece","Leon", "LeeBrickell" ]  :
              control = True
              while control:
                if i == "McEliece":
                    error_McEliece, information_McEliece  = self.decrypt_McEliece(self.G)
                    matrix_McEliece = self.G[:,information_McEliece ] 
                    if self.is_invertible_mod_2(matrix_McEliece):
                        inverse_matrix_McEliece =  Matrix(matrix_McEliece).inv_mod(2)
                        self.recovery_codeword_McEliece =  np.mod((self.ciphertext  -   error_McEliece ), 2) 
                        if np.array_equal(self.recovery_codeword_McEliece, self.original_codeword )!= True:
                            print("Codeword are not equal !")
                            break
                        self.recovery_codeword_McEliece_I = self.recovery_codeword_McEliece[[information_McEliece]]
                        M_inv_mod_2_np = np.array(inverse_matrix_McEliece).astype(int) 
                        result = Matrix(np.mod( np.dot(matrix_McEliece, M_inv_mod_2_np), 2))
                        message_recovered = (np.mod( np.dot(self.recovery_codeword_McEliece_I,inverse_matrix_McEliece  ), 2 ))[0].tolist()
                        if (message_recovered == self.original_message ).all() :
                            print("McEliece\t\t", message_recovered )
                        control = False
                elif i == "Leon":
                    error_Leon, information_Leon = self.decrypt_Leon(self.G)
                    matrix_Leon = self.G[:,information_Leon ]
                    if self.is_invertible_mod_2(matrix_Leon):
                        inverse_matrix_Leon =  Matrix(matrix_Leon).inv_mod(2)
                        self.recovery_codeword_Leon =  np.mod((self.ciphertext  -   error_Leon ), 2) 
                        if np.array_equal(self.recovery_codeword_Leon, self.original_codeword ) !=True:
                            print("Codeword are not equal !")
                            break
                        self.recovery_codeword_Leon_I = self.recovery_codeword_Leon[[information_Leon]]
                        M_inv_mod_2_np = np.array(inverse_matrix_Leon).astype(int) 
                        result = Matrix(np.mod( np.dot(matrix_Leon, M_inv_mod_2_np), 2))
                        message_recovered = (np.mod( np.dot(self.recovery_codeword_Leon_I,inverse_matrix_Leon  ), 2 ))[0].tolist()
                        if (message_recovered == self.original_message ).all() :
                            print("Leon\t\t\t", message_recovered )
                        control = False
                elif i == "LeeBrickell":
                    error_LeeBrickell, information_Brickell = self.decrypt_LeeBrickell(self.G) 
                    matrix_LeeBrickell = self.G[:,information_Brickell ]
                    if self.is_invertible_mod_2(matrix_LeeBrickell):
                        inverse_matrix_LeeBrickell =  Matrix(matrix_LeeBrickell).inv_mod(2)
                        self.recovery_codeword_LeeBrickell =  np.mod((self.ciphertext  -   error_LeeBrickell ), 2) 
                        if np.array_equal(self.recovery_codeword_LeeBrickell, self.original_codeword ) !=True:
                            print("Codeword are not equal !")
                            break
                        self.recovery_codeword_LeeBrickell_I = self.recovery_codeword_Leon[[information_Brickell]]
                        M_inv_mod_2_np = np.array(inverse_matrix_LeeBrickell).astype(int) 
                        result = Matrix(np.mod( np.dot(matrix_LeeBrickell, M_inv_mod_2_np), 2))
                        message_recovered = (np.mod( np.dot(self.recovery_codeword_LeeBrickell_I,inverse_matrix_LeeBrickell  ), 2 ))[0].tolist()
                        if (message_recovered == self.original_message ).all() :
                            print("LeeBrickell\t\t", message_recovered )
                        control = False
                else:
                    pass
    def workfactor(self):
        
        def McEliece():
            aux1 = math.comb(self.n, self.t)
            aux2 = math.comb(int(self.n - self.t) , self.t )
            McEliece_workfactor = round(((aux1 ) / (aux2) )*(self.k / 2)*(self.n*self.k + (self.n - self.k)),3)
            return  McEliece_workfactor
        # Leon
        def W_Leon():
            # Termo constante
            const_term = (self.n * self.k**2 / 2) + (self.n * (self.n - self.k) / 2)
            # Somatório principal no numerador
            sum_numerador = 0
            for r in range(p + 1):
                inner_sum = sum(math.comb(l, i) for i in range(p - r + 1)) / (2**l)
                sum_numerador += math.comb(self.k, r) * (2*l + r*(self.n - self.k - l)) * inner_sum
            # Total do numerador
            numerador = const_term + sum_numerador
            
            # Somatório no denominador
            sum_denominador = 0
            for r in range(p + 1):
                sum_denominador += math.comb(self.n - (self.k + t), t - r) * math.comb(self.k + t, r)
            
            # Normalização do denominador
            denominador = sum_denominador / math.comb(self.n, t)
            
            # Cálculo final de W_Leon
            W = numerador / denominador
            return round(W,3)
        def W_LeeBrickell():
            sum_numerador = sum(math.comb(self.k,j) for j in range(self.p + 1))
            numerador = (self.n * self.k**2 / 2) + (self.n - self.k) * (self.k / 2 + 2 * sum_numerador)
            sum_denominador = sum(math.comb(self.k, r) * math.comb(self.n - self.k, self.t - r) for r in range(self.p + 1))
            W = numerador / (sum_denominador / math.comb(self.n, self.t))
            return round(W,3)
        print("McEliece workfactor:\t", McEliece() )
        print("Leon workfactor:\t",W_Leon())
        print("LeeBrickell workfactor: ",  W_LeeBrickell())
    
    def run(self):
        # encrypt the message
        self.G = self.generate_public_key()
        #self.encrypt(self.G)
        #self.validate()
        self.workfactor()
m, t , p, l = 5, 4, 3, 3
Attacking   =  Attack(m, t, p,l)
Attacking.run()
print()

McEliece workfactor:	 4257.242
Leon workfactor:	 6066.012
LeeBrickell workfactor:  14584.764

